# San Diego County Kindergarten Vaccination Rates Analysis
by [Joe Yerardi](https://twitter.com/JoeYerardi)  

*inewsource* conducted an analysis of data from the [California Department of Education](http://www.cde.ca.gov/ds/sd/sd/filessp.asp) and the [California Department of Public Health](http://www.shotsforschool.org/k-12/reporting-data/) to determine vaccination rates at kindergartens in San Diego County for the 2016-17 school year.

The analysis found that the proportion of San Diego County kindergartners fully up-to-date on their vaccinations reached its highest point in more than a decade and that the gaps between wealthier and poorer schools vanished in the wake of a 2015 law severely curtailing the ability of parents to acquire personal belief exemptions for their children.

These results formed the basis for the April 28, 2017 story "[Kindergartner vaccinations rising in San Diego County after state law change](https://wp.me/p6tuYl-6kJ)."

In [1]:
import geopy
import pandas as pd
import requests
# Suppress scientific notation and set the precision of float values to two
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# Display all columns in dataframes
pd.set_option('display.max_columns', None)
# Display all rows in dataframes
pd.set_option('display.max_rows', None)

## Import kindergarten vaccinations data

We've got some data to import, starting with the Excel spreadsheet of 2016-17 kindergarten vaccinations data.

In [2]:
# Read the school code column as a string, specify the columns to import and assign new column names
vax1617 = pd.read_excel('Data/vaccinations1617.xlsx', converters={'SCHOOL CODE':str}, parse_cols = 'A:V',
                        names=('school_code','county','pub_priv','district','city','school','enrollment',
                               'uptodate','uptodate_pct','uptodate_suppressed','conditional','conditional_pct',
                               'conditional_suppressed','pme','pme_pct','pme_suppressed','pbe','pbe_pct',
                               'pbe_suppressed','others','others_pct','others_suppressed'))
vax1617.info() # 6,469 records, as expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6469 entries, 0 to 6468
Data columns (total 22 columns):
school_code               6469 non-null object
county                    6469 non-null object
pub_priv                  6469 non-null object
district                  5581 non-null object
city                      6469 non-null object
school                    6469 non-null object
enrollment                6469 non-null int64
uptodate                  3183 non-null float64
uptodate_pct              3183 non-null float64
uptodate_suppressed       3286 non-null object
conditional               3183 non-null float64
conditional_pct           3183 non-null float64
conditional_suppressed    3286 non-null object
pme                       3183 non-null float64
pme_pct                   3183 non-null float64
pme_suppressed            3286 non-null object
pbe                       3183 non-null float64
pbe_pct                   3183 non-null float64
pbe_suppressed            3286 non-null 

In [3]:
vax1617.head(1)

,school_code,county,pub_priv,district,city,school,enrollment,uptodate,uptodate_pct,uptodate_suppressed,conditional,conditional_pct,conditional_suppressed,pme,pme_pct,pme_suppressed,pbe,pbe_pct,pbe_suppressed,others,others_pct,others_suppressed
0,6975080,SAN BERNARDINO,PRIVATE,NaN,HIGHLAND,ST. ADELAIDE,49,9.00,18.37,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN


We want to do some year-to-year comparisons between schools so let's also import the 2015-16 kindergarten vaccinations data.

In [4]:
# Read the school code column as a string, specify the columns to import and assign new column names
vax1516 = pd.read_excel('data/vaccinations1516.xlsx', converters={'SCHOOL CODE':str}, parse_cols = 'A:O',
                        names=('school_code','county','pub_priv','district','city','school','enrollment','uptodate',
                        'uptodate_pct','conditional','conditional_pct','pme','pme_pct','pbe','pbe_pct'))
vax1516.info() # 7,422 records, as expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7422 entries, 0 to 7421
Data columns (total 15 columns):
school_code        7422 non-null object
county             7422 non-null object
pub_priv           7422 non-null object
district           5784 non-null object
city               7422 non-null object
school             7422 non-null object
enrollment         7137 non-null float64
uptodate           7137 non-null float64
uptodate_pct       7137 non-null float64
conditional        7137 non-null float64
conditional_pct    7137 non-null float64
pme                7137 non-null float64
pme_pct            7137 non-null float64
pbe                7137 non-null float64
pbe_pct            7137 non-null float64
dtypes: float64(9), object(6)
memory usage: 869.8+ KB


In [5]:
vax1516.head(1)

,school_code,county,pub_priv,district,city,school,enrollment,uptodate,uptodate_pct,conditional,conditional_pct,pme,pme_pct,pbe,pbe_pct
0,0130419,ALAMEDA,PUBLIC,ALAMEDA COUNTY OFFICE OF EDUCATION,HAYWARD,ALAMEDA COUNTY COMMUNITY,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Import school lunch data

Now let's import the Excel spreadsheet of free and reduced price lunch figures.

In [6]:
# Read the school code column as a string, specify the sheet and columns to import, skip the first row after the headers and assign new column names
lunch = pd.read_excel('data/lunch1617.xlsx', sheetname='FRPM School-Level Data ', skiprows=1, converters={'School Code':str},
                      parse_cols = 'D,G,V', names=('school_code','school','frpm_pct'))
lunch.info() # 10,478 records, as expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 3 columns):
school_code    10478 non-null object
school         10478 non-null object
frpm_pct       10478 non-null float64
dtypes: float64(1), object(2)
memory usage: 245.7+ KB


In [7]:
lunch.head(1)

,school_code,school,frpm_pct
0,0112607,Envision Academy for Arts & Technology,0.76


## Merge, filter and analyze the data

Let's start by merging the two years of vaccinations data.

In [8]:
vax = pd.merge(vax1617, vax1516, on='school_code', how='left', suffixes=('_1617', '_1516'))
vax.head(1)

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516
0,6975080,SAN BERNARDINO,PRIVATE,NaN,HIGHLAND,ST. ADELAIDE,49,9.00,18.37,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan


Now let's filter the two years of kindergarten vaccinations data down to just those schools in San Diego County.

In [9]:
vax_sd = vax[vax['county_1617'] == 'SAN DIEGO']
vax_sd.head(1)

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516
2,7080427,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,ISLAMIC SCHOOL OF SAN DIEGO,40,20.00,50.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,ISLAMIC SCHOOL OF SAN DIEGO,24.00,24.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00


And now merge the vaccinations data with the school lunch data.

In [10]:
vax_sd = pd.merge(vax_sd, lunch, on='school_code', how='left', suffixes=('_vax', '_lunch'))
vax_sd.info() # 509 records returned, as expected

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509 entries, 0 to 508
Data columns (total 38 columns):
school_code               509 non-null object
county_1617               509 non-null object
pub_priv_1617             509 non-null object
district_1617             450 non-null object
city_1617                 509 non-null object
school_1617               509 non-null object
enrollment_1617           509 non-null int64
uptodate_1617             269 non-null float64
uptodate_pct_1617         269 non-null float64
uptodate_suppressed       240 non-null object
conditional_1617          269 non-null float64
conditional_pct_1617      269 non-null float64
conditional_suppressed    240 non-null object
pme_1617                  269 non-null float64
pme_pct_1617              269 non-null float64
pme_suppressed            240 non-null object
pbe_1617                  269 non-null float64
pbe_pct_1617              269 non-null float64
pbe_suppressed            240 non-null object
others        

Finally, it's time to analyze.  
  
Our first question: Which schools saw the greatest increase in the proportion of their kindergarteners that are up-to-date on their vaccinations?

In [11]:
vax_sd['pct_change_uptodate'] = ((vax_sd['uptodate_pct_1617'] - vax_sd['uptodate_pct_1516']) / vax_sd['uptodate_pct_1516']) * 100
vax_sd.sort_values('pct_change_uptodate', ascending=False).head()

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516,school,frpm_pct,pct_change_uptodate
148,0127118,SAN DIEGO,PUBLIC,DEHESA ELEMENTARY,ALPINE,THE HEIGHTS CHARTER,35,33.00,94.29,NaN,0.00,0.00,NaN,2.00,5.71,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PUBLIC,DEHESA ELEMENTARY,ALPINE,THE HEIGHTS CHARTER,24.00,3.00,12.50,19.00,79.17,0.00,0.00,2.00,8.33,The Heights Charter,0.20,654.29
79,0108548,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,IFTIN CHARTER,66,58.00,87.88,NaN,8.00,12.12,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,IFTIN CHARTER,46.00,21.00,45.65,24.00,52.17,0.00,0.00,1.00,2.17,Iftin Charter,0.89,92.51
224,6996045,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,MARIA MONTESSORI SCHOOL,21,17.00,80.95,NaN,2.00,9.52,NaN,0.00,0.00,NaN,2.00,9.52,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,MARIA MONTESSORI SCHOOL,34.00,17.00,50.00,15.00,44.12,0.00,0.00,2.00,5.88,NaN,nan,61.90
229,7102098,SAN DIEGO,PRIVATE,NaN,CARLSBAD,SANDERLING WALDORF SCHOOL,21,9.00,42.86,NaN,1.00,4.76,NaN,5.00,23.81,NaN,6.00,28.57,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,CARLSBAD,SANDERLING WALDORF SCHOOL,32.00,9.00,28.13,0.00,0.00,0.00,0.00,23.00,71.88,NaN,nan,52.35
239,6118723,SAN DIEGO,PUBLIC,ALPINE UNION ELEMENTARY,ALPINE,CREEKSIDE EARLY LEARNING CENTER,244,229.00,93.85,NaN,9.00,3.69,NaN,1.00,0.41,NaN,3.00,1.23,NaN,2.00,0.82,NaN,SAN DIEGO,PUBLIC,ALPINE UNION ELEMENTARY,ALPINE,CREEKSIDE EARLY LEARNING CENTER,214.00,137.00,64.02,76.00,35.51,0.00,0.00,1.00,0.47,Creekside Early Learning Center,0.39,46.60


It looks like The Heights Charter in the Dehesa Elementary school district saw the greatest overall increase in the proportion of their kindergarteners that are up-to-date on their vaccinations. How about just the personal belief exemptions?

In [12]:
vax_sd['pct_change_pbe'] = ((vax_sd['pbe_pct_1617'] - vax_sd['pbe_pct_1516']) / vax_sd['pbe_pct_1516']) * 100
vax_sd.sort_values(['pct_change_pbe', 'pbe_pct_1516'], ascending=[True, False]).head()

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516,school,frpm_pct,pct_change_uptodate,pct_change_pbe
153,6203491,SAN DIEGO,PRIVATE,NaN,RANCHO SANTA FE,HORIZON PREP,31,25.00,80.65,NaN,3.00,9.68,NaN,3.00,9.68,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,RANCHO SANTA FE,HORIZON PREP,40.00,27.00,67.50,0.00,0.00,0.00,0.00,13.00,32.50,NaN,nan,19.47,-100.00
266,0106799,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,LA MESA,LEARNING CHOICE ACADEMY,54,29.00,53.70,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,25.00,46.30,NaN,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,LEARNING CHOICE ACADEMY,56.00,33.00,58.93,6.00,10.71,0.00,0.00,17.00,30.36,Learning Choice Academy,0.57,-8.87,-100.00
145,6927883,SAN DIEGO,PRIVATE,NaN,SOLANA BEACH,SANTA FE CHRISTIAN SCHOOLS,38,35.00,92.11,NaN,1.00,2.63,NaN,2.00,5.26,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,SOLANA BEACH,SANTA FE CHRISTIAN SCHOOLS,42.00,29.00,69.05,1.00,2.38,0.00,0.00,12.00,28.57,NaN,nan,33.39,-100.00
159,0118083,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,INNOVATIONS ACADEMY,46,35.00,76.09,NaN,4.00,8.70,NaN,7.00,15.22,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,INNOVATIONS ACADEMY,40.00,26.00,65.00,3.00,7.50,0.00,0.00,11.00,27.50,Innovations Academy,0.31,17.06,-100.00
155,6119168,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,SAN DIEGO COOPERATIVE CHARTER,48,42.00,87.50,NaN,0.00,0.00,NaN,6.00,12.50,NaN,0.00,0.00,NaN,0.00,0.00,NaN,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,SAN DIEGO COOPERATIVE CHARTER,49.00,36.00,73.47,3.00,6.12,0.00,0.00,10.00,20.41,San Diego Cooperative Charter,0.32,19.10,-100.00


Horizon Prep in Rancho Santa Fe saw the biggest decline in personal belief exemptions between the 2015-16 and 2016-17 school year. It went from 13 of 40 students (32.5%) having a personal belief exemption to none.

And precisely how many schools now have no students with personal belief exemptions and how does that compare to the 2015-16 school year?

In [13]:
vax_sd[(vax_sd.pbe_1617 == 0.00)].shape

(177, 40)

And how many schools with suppressed data have at most 5 percent of students with personal belief exemptions?

In [14]:
vax_sd[(vax_sd.pbe_suppressed.str.contains('≤1%', na=False)) | 
       (vax_sd.pbe_suppressed.str.contains('≤2%', na=False)) | 
      (vax_sd.pbe_suppressed.str.contains('≤5%', na=False))].shape

(240, 40)

For comparison, how many schools in the 2015-16 school year had no students with personal belief exemptions?

In [15]:
vax_sd[(vax_sd.pbe_1516 == 0.00)].shape

(154, 40)

Which schools have the lowest proportion of their kindergarteners that are up-to-date on their vaccinations?

In [16]:
vax_sd.sort_values('uptodate_pct_1617', ascending=True).head()

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516,school,frpm_pct,pct_change_uptodate,pct_change_pbe
265,0132886,SAN DIEGO,PUBLIC,BONSALL UNIFIED,BONSALL,PATHWAYS ACADEMY CHARTER,40,3.00,7.50,NaN,12.00,30.00,NaN,1.00,2.50,NaN,8.00,20.00,NaN,16.00,40.00,NaN,SAN DIEGO,PUBLIC,BONSALL UNION ELEMENTARY,BONSALL,PATHWAYS CHARTER,136.00,75.00,55.15,19.00,13.97,0.00,0.00,42.00,30.88,Pathways Academy Charter,0.41,-86.40,-35.23
268,0127084,SAN DIEGO,PUBLIC,MOUNTAIN EMPIRE UNIFIED,THOUSAND OAKS,ACADEMY OF ARTS AND SCIENCES: DEL MAR MIDDLE &...,22,4.00,18.18,NaN,0.00,0.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,18.00,81.82,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,Academy of Arts and Sciences,0.62,nan,nan
267,7052608,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,HERITAGE CHRISTIAN SCHOOL OF SAN DIEGO,30,7.00,23.33,NaN,0.00,0.00,NaN,2.00,6.67,NaN,2.00,6.67,NaN,19.00,63.33,NaN,SAN DIEGO,PRIVATE,NaN,SAN DIEGO,HERITAGE CHRISTIAN SCHOOL OF SAN DIEGO,27.00,21.00,77.78,0.00,0.00,0.00,0.00,6.00,22.22,NaN,nan,-70.00,-70.00
264,0132472,SAN DIEGO,PUBLIC,WARNER UNIFIED,WARNER SPRINGS,CALIFORNIA PACIFIC CHARTER SCHOOLS - SAN DIEGO,260,92.00,35.38,NaN,26.00,10.00,NaN,4.00,1.54,NaN,50.00,19.23,NaN,88.00,33.85,NaN,SAN DIEGO,PUBLIC,WARNER UNIFIED,WARNER SPRINGS,CALIFORNIA PACIFIC CHARTER SCHOOL - SD,78.00,25.00,32.05,3.00,3.85,0.00,0.00,50.00,64.10,California Pacific Charter Schools - San Diego,0.26,10.40,-70.00
230,0132506,SAN DIEGO,PUBLIC,DEHESA ELEMENTARY,EL CAJON,INSPIRE CHARTER SCHOOL - SOUTH,525,210.00,40.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,315.00,60.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inspire Charter School - South,0.33,nan,nan


So only 7.5% of the kindergarteners at Pathways Academy Charter in the Bonsall Unified school distrct are up-to-date on their vaccinations. Now what if we look at those schools where the exact number of students has been suppressed? Are any below 7.5%?

In [17]:
vax_sd.sort_values('uptodate_suppressed', ascending=True).head()

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516,school,frpm_pct,pct_change_uptodate,pct_change_pbe
447,6150312,SAN DIEGO,PRIVATE,NaN,CHULA VISTA,MATER DEI JUAN DIEGO ACADEMY,30,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan
508,6975973,SAN DIEGO,PRIVATE,NaN,VISTA,VISTA CHRISTIAN,23,nan,nan,≥95%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,SAN DIEGO,PRIVATE,NaN,VISTA,VISTA CHRISTIAN,25.00,20.00,80.00,3.00,12.00,0.00,0.00,2.00,8.00,NaN,nan,nan,nan
474,6040117,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,ROWAN ELEMENTARY,40,nan,nan,≥95%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,ROWAN ELEMENTARY,45.00,44.00,97.78,1.00,2.22,0.00,0.00,0.00,0.00,Rowan Elementary,0.80,nan,nan
473,0123778,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,OLD TOWN ACADEMY K-8 CHARTER,28,nan,nan,≥95%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,OLD TOWN ACADEMY K-8 CHARTER,28.00,16.00,57.14,0.00,0.00,0.00,0.00,2.00,7.14,Old Town Academy K-8 Charter,0.10,nan,nan
472,6119598,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,KING-CHAVEZ ACADEMY OF EXCELLENCE,24,nan,nan,≥95%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,nan,nan,≤5%,SAN DIEGO,PUBLIC,SAN DIEGO UNIFIED,SAN DIEGO,KING-CHAVEZ ACADEMY OF EXCELLENCE,24.00,24.00,100.00,0.00,0.00,0.00,0.00,0.00,0.00,King-Chavez Academy of Excellence,0.98,nan,nan


Yes, there is one school with an up-to-date percentage less than or equal to five percent: Mater Dei Juan Diego Academy. The private school in Chula Vista has an enrollment of 30 students.

And which school has the highest proportion of students with personal belief exemptions?

In [18]:
vax_sd.sort_values('pbe_pct_1617', ascending=False).head()

,school_code,county_1617,pub_priv_1617,district_1617,city_1617,school_1617,enrollment_1617,uptodate_1617,uptodate_pct_1617,uptodate_suppressed,conditional_1617,conditional_pct_1617,conditional_suppressed,pme_1617,pme_pct_1617,pme_suppressed,pbe_1617,pbe_pct_1617,pbe_suppressed,others,others_pct,others_suppressed,county_1516,pub_priv_1516,district_1516,city_1516,school_1516,enrollment_1516,uptodate_1516,uptodate_pct_1516,conditional_1516,conditional_pct_1516,pme_1516,pme_pct_1516,pbe_1516,pbe_pct_1516,school,frpm_pct,pct_change_uptodate,pct_change_pbe
230,0132506,SAN DIEGO,PUBLIC,DEHESA ELEMENTARY,EL CAJON,INSPIRE CHARTER SCHOOL - SOUTH,525,210.00,40.00,NaN,0.00,0.00,NaN,0.00,0.00,NaN,315.00,60.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,Inspire Charter School - South,0.33,nan,nan
229,7102098,SAN DIEGO,PRIVATE,NaN,CARLSBAD,SANDERLING WALDORF SCHOOL,21,9.00,42.86,NaN,1.00,4.76,NaN,5.00,23.81,NaN,6.00,28.57,NaN,0.00,0.00,NaN,SAN DIEGO,PRIVATE,NaN,CARLSBAD,SANDERLING WALDORF SCHOOL,32.00,9.00,28.13,0.00,0.00,0.00,0.00,23.00,71.88,NaN,nan,52.35,-60.25
265,0132886,SAN DIEGO,PUBLIC,BONSALL UNIFIED,BONSALL,PATHWAYS ACADEMY CHARTER,40,3.00,7.50,NaN,12.00,30.00,NaN,1.00,2.50,NaN,8.00,20.00,NaN,16.00,40.00,NaN,SAN DIEGO,PUBLIC,BONSALL UNION ELEMENTARY,BONSALL,PATHWAYS CHARTER,136.00,75.00,55.15,19.00,13.97,0.00,0.00,42.00,30.88,Pathways Academy Charter,0.41,-86.40,-35.23
264,0132472,SAN DIEGO,PUBLIC,WARNER UNIFIED,WARNER SPRINGS,CALIFORNIA PACIFIC CHARTER SCHOOLS - SAN DIEGO,260,92.00,35.38,NaN,26.00,10.00,NaN,4.00,1.54,NaN,50.00,19.23,NaN,88.00,33.85,NaN,SAN DIEGO,PUBLIC,WARNER UNIFIED,WARNER SPRINGS,CALIFORNIA PACIFIC CHARTER SCHOOL - SD,78.00,25.00,32.05,3.00,3.85,0.00,0.00,50.00,64.10,California Pacific Charter Schools - San Diego,0.26,10.40,-70.00
259,6120893,SAN DIEGO,PUBLIC,SPENCER VALLEY ELEMENTARY,SIMI VALLEY,CALIFORNIA VIRTUAL ACADEMY @ SAN DIEGO,73,49.00,67.12,NaN,0.00,0.00,NaN,4.00,5.48,NaN,14.00,19.18,NaN,6.00,8.22,NaN,SAN DIEGO,PUBLIC,SPENCER VALLEY ELEMENTARY,SIMI VALLEY,CALIFORNIA VIRTUAL ACADEMY SAN DIEGO,108.00,90.00,83.33,0.00,0.00,0.00,0.00,18.00,16.67,California Virtual Academy @ San Diego,0.58,-19.45,15.05


How many kindergartens saw an increase in the proportion of their students that are up-to-date on their vaccinations?

In [19]:
vax_sd[vax_sd.pct_change_uptodate > 0].shape

(141, 40)

And how many kindergartens saw a decrease?

In [20]:
vax_sd[vax_sd.pct_change_uptodate < 0].shape

(121, 40)

In [21]:
vax_sd.to_excel('vax_sd.xlsx')

When we analyzed the data two years ago, we found that wealthier schools (as measured by public vs. private and the proportion of students receiving free-and-reduced-price lunch) had lower personal belief exemption rates than poorer schools. Is that still the case?  
  
To find out, let's create two new dataframes, one of the public kindergartens and one of the private kindergartens, and then calcuate the personal belief exemption rates for each set of data. We'll also need to exclude those kindergartens where data has been partially masked.

In [22]:
vax_sd_pub = vax_sd[(vax_sd.pub_priv_1617 == 'PUBLIC') & (vax_sd.pbe_1617.notnull())]
vax_sd_priv = vax_sd[(vax_sd.pub_priv_1617 == 'PRIVATE') & (vax_sd.pbe_1617.notnull())] 

In [23]:
# Calculate the PBE exemption rate for public schools
vax_sd_pub.pbe_1617.sum() / vax_sd_pub.enrollment_1617.sum()

0.02385507836352534

In [24]:
# Calculate the PBE exemption rate for private schools
vax_sd_priv.pbe_1617.sum() / vax_sd_priv.enrollment_1617.sum()

0.025763358778625955

So the rates for public and private schools are similar. What about when we look at the rates for the proportion of students receiving free-and-reduced-price lunch?  
  
The first step is to create two new dataframes, one of the public kindergartens where more than half of the student population qualified for free or reduced-cost meals and one of the public kindergartens where half or less of the student population qualified for free or reduced-cost meals.

In [25]:
vax_sd_high_frpm = vax_sd_pub[vax_sd_pub.frpm_pct > 0.5]
vax_sd_low_frpm = vax_sd_pub[vax_sd_pub.frpm_pct <= 0.5]

In [26]:
# Calculate the PBE exemption rate for high FRPM kindergartens
vax_sd_high_frpm.pbe_1617.sum() / vax_sd_high_frpm.enrollment_1617.sum()

0.004946219674962707

In [27]:
# Calculate the PBE exemption rate for low FRPM kindergartens
vax_sd_low_frpm.pbe_1617.sum() / vax_sd_low_frpm.enrollment_1617.sum()

0.04421711193777477

## Import school address data

Now that we've analyzed the latest year of data, we need to prepare it for mapping. Let's start by bring in the Excel spreadsheets of school addresses, starting with the public schools file.

In [28]:
# Read the CDS code, zip code, latitude and longitude columns as strings, specify the columns to import and assign new column names
pubaddress = pd.read_excel('data/pubschools.xlsx', converters={'CDSCode':str, 'Zip':str, 'Latitude':str,'Longitude':str}, parse_cols='A,G:H,J:L,AL:AM',
                           names=('cds_code','school','street','city','zip','state','latitude','longitude'))
pubaddress.info() # 17,707 records returned, as expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17707 entries, 0 to 17706
Data columns (total 8 columns):
cds_code     17707 non-null object
school       16337 non-null object
street       17415 non-null object
city         17415 non-null object
zip          17415 non-null object
state        17415 non-null object
latitude     12899 non-null object
longitude    12899 non-null object
dtypes: object(8)
memory usage: 1.1+ MB


In [29]:
pubaddress.head(1)

,cds_code,school,street,city,zip,state,latitude,longitude
0,01100170000000,NaN,313 West Winton Avenue,Hayward,94544-1136,CA,37.658212,-122.09713


Now we need to slice out the last seven digits of the CDS code to get the school code.

In [30]:
pubaddress['school_code'] = pubaddress['cds_code'].str[-7:]
pubaddress.drop('cds_code', axis=1, inplace=True) # Delete the CDS code column as it is no longer needed
pubaddress.head()

,school,street,city,zip,state,latitude,longitude,school_code
0,NaN,313 West Winton Avenue,Hayward,94544-1136,CA,37.658212,-122.09713,0000000
1,FAME Public Charter,"39899 Balentine Drive, Suite 335",Newark,94560-5359,CA,37.521436,-121.99391,0109835
2,Envision Academy for Arts & Technology,1515 Webster Street,Oakland,94612-3355,CA,37.804520,-122.26815,0112607
3,Aspire California College Preparatory Academy,2125 Jefferson Avenue,Berkeley,94703-1414,CA,37.868991,-122.27844,0118489
4,Community School for Creative Education,2111 International Boulevard,Oakland,94606-4903,CA,37.784648,-122.23863,0123968


And finally, let's merge the street, city, state and zip columns into a single address column on which we will eventually map the data.

In [31]:
pubaddress['address'] = pubaddress['street'] + ' ' + pubaddress['city'] + ', ' + pubaddress['state'] + ' ' + pubaddress['zip']
pubaddress.drop(['street', 'city',  'state', 'zip'], axis=1, inplace=True) # Delete the street, city, state and zip columns as they are no longer needed
pubaddress.head()

,school,latitude,longitude,school_code,address
0,NaN,37.658212,-122.09713,0000000,"313 West Winton Avenue Hayward, CA 94544-1136"
1,FAME Public Charter,37.521436,-121.99391,0109835,"39899 Balentine Drive, Suite 335 Newark, CA 94..."
2,Envision Academy for Arts & Technology,37.804520,-122.26815,0112607,"1515 Webster Street Oakland, CA 94612-3355"
3,Aspire California College Preparatory Academy,37.868991,-122.27844,0118489,"2125 Jefferson Avenue Berkeley, CA 94703-1414"
4,Community School for Creative Education,37.784648,-122.23863,0123968,"2111 International Boulevard Oakland, CA 94606..."


And now let's import the addresses for the private schools.

In [32]:
# Read the CDS code and zip code columns as strings, specify the columns to import and assign new column names
privaddress = pd.read_excel('data/privschools1617.xlsx', converters={'CDS Code': str, 'Zip': str}, 
                            parse_cols='C:H', names=('cds_code', 'school', 'street', 'city', 'state', 'zip'))
privaddress.info() # 2,939 records returned, as expected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2939 entries, 0 to 2938
Data columns (total 6 columns):
cds_code    2772 non-null object
school      2939 non-null object
street      2939 non-null object
city        2939 non-null object
state       2939 non-null object
zip         2939 non-null object
dtypes: object(6)
memory usage: 137.8+ KB


In [33]:
privaddress.head(1)

,cds_code,school,street,city,state,zip
0,01612596910343,Agnes Memorial Christian Academey,2372 International Boulevard,Oakland,CA,94601


As with the public schools file, we need to slice out the last seven digits of the CDS code to get the school code.

In [34]:
privaddress['school_code'] = privaddress['cds_code'].str[-7:]
privaddress.drop('cds_code', axis=1, inplace=True) # Delete the CDS code column as it is no longer needed
privaddress.head()

,school,street,city,state,zip,school_code
0,Agnes Memorial Christian Academey,2372 International Boulevard,Oakland,CA,94601,6910343
1,All Saints Catholic School,22870 Second Street,Hayward,CA,94541,6965909
2,Alsion Montessori Middle/High,750 Witherly Lane,Fremont,CA,94539,7103799
3,American International Montessori School,3339 Martin Luther King Jr. Way,Berkeley,CA,94703,6135743
4,"Anchor Education, Inc. dba Sterling West -- Sa...",19234 Lake Chabot Road,Castro Valley,CA,94577,6133441


And finally, let's merge the street, city, state and zip columns into a single address column on which we will eventually map the data.

In [35]:
privaddress['address'] = privaddress['street'] + ' ' + privaddress['city'] + ', ' + privaddress['state'] + ' ' + privaddress['zip']
privaddress.drop(['street', 'city',  'state', 'zip'], axis=1, inplace=True) # Delete the street, city, state and zip columns as they are no longer needed
privaddress.head()

,school,school_code,address
0,Agnes Memorial Christian Academey,6910343,"2372 International Boulevard Oakland, CA 94601"
1,All Saints Catholic School,6965909,"22870 Second Street Hayward, CA 94541"
2,Alsion Montessori Middle/High,7103799,"750 Witherly Lane Fremont, CA 94539"
3,American International Montessori School,6135743,"3339 Martin Luther King Jr. Way Berkeley, CA 9..."
4,"Anchor Education, Inc. dba Sterling West -- Sa...",6133441,"19234 Lake Chabot Road Castro Valley, CA 94577"


### Concatenate, join and export the data

Now we need to concatenate (combine) the public and private school address dataframes so we can then join them with the 2016-17 vaccinations/lunch dataframe.

In [36]:
all_addresses = pd.concat([pubaddress, privaddress])
all_addresses.info() # 20,646 records returned, as expected

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20646 entries, 0 to 2938
Data columns (total 5 columns):
address        20354 non-null object
latitude       12899 non-null object
longitude      12899 non-null object
school         19276 non-null object
school_code    20479 non-null object
dtypes: object(5)
memory usage: 967.8+ KB


Merge the addresses and vaccinations/school lunch dataframes.

In [37]:
vax_sd_addresses = pd.merge(vax_sd, all_addresses, on='school_code', how='left', suffixes=('_vax', '_addresses'))
vax_sd_addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511 entries, 0 to 510
Data columns (total 44 columns):
school_code               511 non-null object
county_1617               511 non-null object
pub_priv_1617             511 non-null object
district_1617             452 non-null object
city_1617                 511 non-null object
school_1617               511 non-null object
enrollment_1617           511 non-null int64
uptodate_1617             269 non-null float64
uptodate_pct_1617         269 non-null float64
uptodate_suppressed       242 non-null object
conditional_1617          269 non-null float64
conditional_pct_1617      269 non-null float64
conditional_suppressed    242 non-null object
pme_1617                  269 non-null float64
pme_pct_1617              269 non-null float64
pme_suppressed            242 non-null object
pbe_1617                  269 non-null float64
pbe_pct_1617              269 non-null float64
pbe_suppressed            242 non-null object
others        

And finally, export the data as an Excel file.

In [38]:
vax_sd_addresses.to_excel('data/vax_sd_addresses.xlsx')